## The `chatlas` package for a consistent LLM interface and workflow

One of the frustrations of working with different LLM providers is the difference in API structures.  This has historically meant that developers have had to code up their LLM workflows quite differently if they were working with, say, OpenAI versus Anthropic or Llama.

The `chatlas` package attempts to overcome this by offering a set of classes and methods that have greater alignment across LLM providers.  This means that starting a chat, or using tool calling or other services will look the same or very similar, no matter which LLM provider you are using.  

The following is a short illustratory demo of some of the features of `chatlas`.

### Starting a chat session - OpenAI example

In [ ]:
# import packages and key environment variables
from dotenv import load_dotenv
from chatlas import ChatOpenAI, ChatAnthropic, ChatOllama
import os

load_dotenv()
INSTANCE_ID = os.getenv('INSTANCE_ID')
API_KEY = os.getenv('API_KEY')
BASE_URL_STEM = os.getenv('BASE_URL_STEM')

In [ ]:
# set up an openai chat client
PROVIDER = "openai"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}/v1"

chat = ChatOpenAI(
    model = "gpt-4o",
    api_key = API_KEY,
    base_url = BASE_URL,
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
# start a chat app
chat.app()

In [ ]:
# chat in the console
chat.console()

In [ ]:
# programmatic chat
chat.chat("What exactly is a spirit vegetable?")

### Tool (function) calling - OpenAI example

In [ ]:
# tool (function) calling - function to get current temperature
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  """Get the current temperature in a given place."""
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

In [ ]:
# register the function with your chat
chat.register_tool(get_current_temperature)

In [ ]:
# now the chat can use it
chat.chat("I'm in Atlanta today and I'm told I should wear warm clothes.  What do you think?")

### Structured data extraction - OpenAI example

In [ ]:
# pulling structured data out of text
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    pets: int
    skills: list[str]

chat.extract_data(
  "My name is Keith.  I have two cats and one dog named Bertie.  I am very good at Math and Computer Games", 
  data_model=Person,
)

### Start a new chat - Anthropic example

In [ ]:
# now try a Anthropic chat client- note similar but not 100% identical to OpenAI
PROVIDER = "anthropic"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}"

chat = ChatAnthropic(
    api_key = API_KEY,
    system_prompt = "You are a friendly but terse assistant.",
    kwargs = {"base_url": BASE_URL}  
)

In [ ]:
# programmatic chat
chat.chat("Which integer is commonly quoted as the answer to the meaning of life?")

In [ ]:
# register tool
chat.register_tool(get_current_temperature)

In [ ]:
# check tool use
chat.chat("My sister is heading to the capital of Norway next week?  How should she pack?")

In [ ]:
#  extract structured data
class Person(BaseModel):
    name: str
    pets: int
    areas_of_expertise: list[str]
    qualifications: list[str]

chat.extract_data(
  """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

In [ ]:
# update structured information during chat session
chat.extract_data(
  """
  Oh it's Keith again.  I'm sorry, I'm so dumb.  I forgot to mention I also have a new puppy which we only just picked up this week.
  """, 
  data_model = Person,
)

In [ ]:
# update again
chat.extract_data(
  """
  It's Keith again.  I just spoke to my wife and she left the door open and the puppy ran out on the street and got killed.  It's a bad day.
  """, 
  data_model = Person,
)

### New chat session with local model using `ollama`

In [ ]:
# local ollama model
chat = ChatOllama(
    model = "llama3.1:8b",
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
# programmatic chat
chat.chat("Explain Buddhism in two sentences")

In [ ]:
# extract data using earlier data model
chat.extract_data(
      """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

In [ ]:
# register temperature tool
chat.register_tool(get_current_temperature)

In [ ]:
# use temperature tool
chat.chat("I'm heading Tasmania tomorrow but I forgot my sunscreen.  Should I be concerned?")